In [1]:
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import sqlite3
import numpy as np
from lsst.sims.maf.metrics.snSLMetric import SNSLMetric
import lsst.sims.maf.metrics as metrics
import healpy as hp
import os

import matplotlib.pylab as plt
%matplotlib inline


In [2]:
path_where_version_folder_is_stored = "dbs"
version = 'v1.5' # "v1.5"

list_of_databases = os.listdir("%s"% (path_where_version_folder_is_stored))


In [4]:
for i in list_of_databases:

    #if i == "baseline_nexp1_v1.6_10yrs.db": 

    # Database available from https://lsst-web.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.4/baseline/
    dbFile = os.path.join(path_where_version_folder_is_stored,'%s' % (i)) #'descddf_v1.4_10yrs.db'
    dbName = i.replace('.db', '')
    opsimdb = db.OpsimDatabase(dbFile)
    
    
    slicer = slicers.HealpixSlicer(nside=64, useCache=False)
    
            
    bundleList= []
    
    metric = SNSLMetric(night_collapse=True)
    summaryStats = [metrics.SumMetric()]            
    sql = '' 
    
    print(metric)
    
    bundleList.append(metricBundles.MetricBundle(
        metric, slicer, sql, runName= dbName,summaryMetrics=summaryStats))

    if version == "v1.7":
        outDir = 'SL_fbs17/%s_SLTimeDelay' % (dbName)      
    elif version == "v1.6":
        outDir = 'SL_fbs16/%s_SLTimeDelay' % (dbName)    
    elif version == "v1.5":
        outDir = 'SL_fbs15/%s_SLTimeDelay' % (dbName)
    elif version == "v1.4":
        outDir = 'SL_fbs14/%s_SLTimeDelay' % (dbName)
    resultsDb = db.ResultsDb(outDir=outDir)
    bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
    bgroup = metricBundles.MetricBundleGroup(bundleDict, opsimdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll()
    
    bgroup.plotAll(closefigs=False)
    
    strategy_name = list(bundleDict)[0]
    
    nSL = bundleDict[strategy_name]
    nSL = nSL.metricValues[~nSL.metricValues.mask]
    nSL = nSL[nSL>0]    
    print("%s: %f" % (i,np.sum(nSL)))
    
    
    hp.mollview(bundleList[0].metricValues,  title='sum= %f' % bundleList[0].summaryValues['Sum'])

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Querying database SummaryAllProps with no constraint for columns ['observationId', 'fieldDec', 'night', 'filter', 'visitTime', 'visitExposureTime', 'fiveSigmaDepth', 'observationStartMJD', 'fieldRA', 'numExposures'].
Found 2056927 visits
Running:  ['baseline_2snaps_v1_5_10yrs_SNSLMetric_HEAL']


/Users/yoachim/git_repos/sims_maf/python/lsst/sims/maf/maps/dustMap.py:24: UserWarning: Slicer value of nside (64) different from map value (128), using slicer value
  warnings.warn('Slicer value of nside (%i) different from map value (%i), using slicer value'


KeyboardInterrupt: 